In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import random
random.seed(2025)

import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras

from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Input, Flatten
from keras.models import Sequential, Model
from keras.applications import VGG16, DenseNet121, EfficientNetB0,ResNet50

In [ ]:
os.listdir('drive/MyDrive/CapstoneProject2025_2/dataset')

['positivo_224', 'negativo_224']

In [ ]:
clases = ['positivo_224','negativo_224']
diccionario = {}
for c in clases:
  diccionario[c] = []
  ruta = 'drive/MyDrive/CapstoneProject2025_2/dataset/'+c
  for imagen in os.listdir(ruta):
    imagen = cv.imread(ruta+'/'+imagen)
    diccionario[c].append(imagen)

In [ ]:
print(len(diccionario['positivo_224']))
print(len(diccionario['negativo_224']))

4009
3432


In [ ]:
len(diccionario['positivo_224'])

4009

In [ ]:
X_pos = random.sample(diccionario['positivo_224'],3432)
random.shuffle(X_pos)
X_neg = diccionario['negativo_224']
random.shuffle(X_neg)
X = np.array(X_pos + X_neg)

In [ ]:
target = np.array([1]*3432 + [0]*3432)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=2025)

In [ ]:
X_train.shape, y_train.shape

((5491, 224, 224, 3), (5491,))

# **Arquitectura 1**

In [ ]:
modeloDL = Sequential(name='ModeloCell')
modeloDL.add(Input(shape=(224,224,3)))
modeloDL.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
modeloDL.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
modeloDL.add(MaxPool2D())
modeloDL.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
modeloDL.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
modeloDL.add(MaxPool2D())
modeloDL.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
modeloDL.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
modeloDL.add(MaxPool2D())
modeloDL.add(Flatten())
modeloDL.add(Dense(128,activation='sigmoid'))
modeloDL.add(Dropout(0.8))
modeloDL.add(Dense(64,activation='sigmoid'))
modeloDL.add(Dropout(0.8))
modeloDL.add(Dense(2,activation='softmax'))

In [ ]:
modeloDL.compile(optimizer='adam',
                loss=keras.losses.sparse_categorical_crossentropy,
                metrics=[keras.metrics.sparse_categorical_accuracy])

In [ ]:
modeloDL.fit(X_train,y_train, epochs=5, batch_size=100)

Epoch 1/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 75s 791ms/step - loss: 1.0892 - sparse_categorical_accuracy: 0.5008
Epoch 2/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 25s 285ms/step - loss: 0.8622 - sparse_categorical_accuracy: 0.4990
Epoch 3/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 16s 289ms/step - loss: 0.7784 - sparse_categorical_accuracy: 0.5001
Epoch 4/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 16s 291ms/step - loss: 0.7388 - sparse_categorical_accuracy: 0.4914
Epoch 5/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 16s 288ms/step - loss: 0.7263 - sparse_categorical_accuracy: 0.4846


# **Arquitectura 2**

In [ ]:
modeloVGG0 = VGG16()

X = modeloVGG0.layers[-3].output
X = Flatten()(X)
X = Dense(2048,activation='relu')(X)
X = Dense(2048,activation='relu')(X)
X = Dense(2,activation='softmax')(X)

nuevoModeloDL = Model(inputs=modeloVGG0.input, outputs = X)

for capa in nuevoModeloDL.layers[:-3]:
  capa.trainable = False

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
nuevoModeloDL.compile(optimizer='adam',
                loss=keras.losses.sparse_categorical_crossentropy,
                metrics=[keras.metrics.sparse_categorical_accuracy])

In [ ]:
nuevoModeloDL.fit(X_train,y_train, epochs=5, batch_size=100)

Epoch 1/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 25s 456ms/step - loss: 0.1621 - sparse_categorical_accuracy: 0.9399
Epoch 2/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 26s 463ms/step - loss: 0.1533 - sparse_categorical_accuracy: 0.9436
Epoch 3/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 25s 451ms/step - loss: 0.1424 - sparse_categorical_accuracy: 0.9459
Epoch 4/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 25s 453ms/step - loss: 0.1263 - sparse_categorical_accuracy: 0.9563
Epoch 5/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 25s 457ms/step - loss: 0.1126 - sparse_categorical_accuracy: 0.9649


In [ ]:
#Formato Keras
nuevoModeloDL.save("drive/MyDrive/CapstoneProject2025_2/modeloCell.keras")

# para leeer
#modelo_ = keras.models.load_model("modeloDL.keras")


#Formato h5
nuevoModeloDL.save("drive/MyDrive/CapstoneProject2025_2/modeloCell.h5", save_format="h5")

# para leeer
#modelo_ = keras.models.load_model("modeloDL.keras")